## Bulk2Space Tutorial
Jie Liao,  Jingyang Qian, Yin Fang, Zhuo Chen, Xiang Zhuang et al.

## Outline
1. [Installation](#Installation)
2. [Import modules](#Import-modules)
3. [Parameter definition](#Parameter-definition)
4. [Load data](#Load-data)
5. [Calculate marker genes of each celltype](#Marker-used)
6. [Data processing](#Data-processing)
7. [Celltype ratio calculation](#Celltype-ratio-calculation)
8. [Prepare the model input](#Prepare-the-model-input)
9. [Model training/loading](#Model-training/loading)
10. [Data generation](#Data-generation)
11. [Data saving](#Data-saving)
12. [Data mapping](#12-a-id"data-mapping"mapping-generated-single-cells-to-spatial-locationsa)


### 1. <a id="Installation">Installation</a>
The installation should take a few minutes on a normal computer. To install Bulk2Space package you must make sure that your python version is over `3.8`. If you don’t know the version of python you can check it by:

In [1]:
import platform
print(platform.python_version())

3.7.10


Note: Because our Bulk2Space dpends on pytorch, you'd better make sure the torch is correctly installed.

### 2. <a id="Import-modules">Import modules</a>

In [2]:
from utils.tool import *
from utils.config import cfg, loadArgums
import numpy as np
import pandas as pd
import torch
import scanpy
from scipy.optimize import nnls
from collections import defaultdict
import argparse
import warnings
warnings.filterwarnings('ignore')

### 3. <a id="Parameter-definition">Parameter definition</a>
For the current version of Bulk2Space,

some parameters should be revised  according to the actual running environment and file Hierarchy:

- `gpu_id`: The GPU ID, eg:`--gpu_id 0`
- `project_name`: The name of your project, eg:`--project_name experiment1`
- `input_bulk_path`: The name of the input bulk-seq data, eg:`--input_bulk_path bulk_data.csv`
- `input_sc_data_path`: The name of the input scRNA-seq data, eg:`--input_sc_data_path sc_data.csv`
- `input_sc_meta_path`: The name of the input scRNA-seq meta, eg:`--input_sc_meta_path sc_meta.csv`
- `input_st_data_path`: The name of the input spatial transcriptomics data, eg:`--input_st_data_path st_data.csv`
- `input_st_meta_path`: The name of the input spatial transcriptomics meta, eg:`--input_st_meta_path st_meta.csv`
- `load_model_1`: Whether to load the trained bulk-deconvolution model, eg:`--load_model_1 False`
- `load_path_1`: The path of the trained bulk-deconvolution model to be loaded
- `train_model_2`: Whether to train the spatial mapping model, eg:`--train_model_2 True`
- `load_path_2`: The path of the trained spatial mapping model to be loaded
- `output_path`: The name of the folder where you store the output data, eg:`--output_path output_data`


some parameters could be revised  as needed:
- `BetaVAE_H`: Whether to use β-VAE model or not, eg:`--BetaVAE_H`
- `batch_size`: The batch size for β-VAE/VAE model training, eg:`--batch_size 512`
- `learning_rate`: The learning rate for β-VAE/VAE model training, eg:`--learning_rate 0.0001`
- `hidden_size`: The hidden size of β-VAE/VAE model, eg:`--hidden_size 256`
- `hidden_lay`: The hidden layer of β-VAE/VAE model(0:[2048, 1024, 512] \n 1: [4096, 2048, 1024, 512] \n 2: [8192, 4096, 2048, 1024]), eg:`--hidden_lay 0`
- `epoch_num`: The epoch number for β-VAE/VAE model training, eg:`--epoch_num 5000`
- `not_early_stop`: Whether to use the `early_stop` strategy, eg:`--not_early_stop False`
- `early_stop`: The model waits N epochs before stops if no progress on the validation set or the training loss dose not decline, eg:`--early_stop 50`
- `k`: The number of cells per spot set in spatial mapping step, eg:`--k 10`
- `marker_used`: Whether to only use marker genes of each celltype when calculating the celltype proportion, eg:`--marker_used True`
- `top_marker_num`: The number of marker genes of each celltype used, eg:`--top_marker_num 500`
- `ratio_num`: The multiples of the number of cells of generated scRNA-seq data, eg:`--ratio_num 1`
- `spot_data`: The type of the input spatial transcriptomics data, `True` for barcoded-based ST data (like ST, 10x Visium or Slide-seq) and  `False` for image-based ST data (like MERFISH, SeqFISH or STARmap)


In [3]:
global args 
args = dict(
    BetaVAE_H=True,
    batch_size=512,
    cell_num=10,
    data_path='example_data/demo1',
    dump_path='/data/zhuangxiang/code/bulk2space/bulk2space/dump',
    early_stop=50,
    epoch_num=10,
    exp_id='LR_0.0001_hiddenSize_256_lay_choice_0',
    exp_name='test1',
    feature_size=6588,
    gpu_id=-1,
    hidden_lay=0,
    hidden_size=256,
    input_bulk_path='/data/zhuangxiang/code/bulk2space/bulk2space/data/example_data/demo1/demo1_bulk.csv',
    input_sc_data_path='/data/zhuangxiang/code/bulk2space/bulk2space/data/example_data/demo1/demo1_sc_data.csv',
    input_sc_meta_path='/data/zhuangxiang/code/bulk2space/bulk2space/data/example_data/demo1/demo1_sc_meta.csv',
    input_st_data_path='/data/zhuangxiang/code/bulk2space/bulk2space/data/example_data/demo1/demo1_st_data.csv',
    input_st_meta_path='/data/zhuangxiang/code/bulk2space/bulk2space/data/example_data/demo1/demo1_st_meta.csv',
    k=10,
    kl_loss=False,
    learning_rate=0.0001,
    load_model_1=False,
    load_path_1='/data/zhuangxiang/code/bulk2space/bulk2space/save_model/',
    load_path_2='/data/zhuangxiang/code/bulk2space/bulk2space/save_model/',
    marker_used=True,
    max_cell_in_diff_spot_ratio=None,
    model_choice_1='vae', model_choice_2='df',
    mul_test=5,
    mul_train=1,
    no_tensorboard=False,
    not_early_stop=False,
    num_workers=12,
    output_path='/data/zhuangxiang/code/bulk2space/bulk2space/output_data',
    previous_project_name='demo', project_name='test1',
    random_seed=12345, ratio_num=1,
    save='/data/zhuangxiang/code/bulk2space/bulk2space/save_model',
    spot_data=True, spot_num=500,
    top_marker_num=500, train_model_2=True,
    xtest='xtest', xtrain='xtrain', ytest='ytest', ytrain='ytrain'
)
args = argparse.Namespace(**args)

In [4]:
used_device = torch.device(f"cuda:{args.gpu_id}") if args.gpu_id >= 0 and torch.cuda.is_available() else torch.device('cpu')

In [5]:
input_sc_meta_path = args.input_sc_meta_path
input_sc_data_path = args.input_sc_data_path
input_bulk_path = args.input_bulk_path
input_st_meta_path = args.input_st_meta_path
input_st_data_path = args.input_st_data_path

### 4. <a id="Load-data">Load data</a>
`Bulk2Space` requires five formatted data as input:
- Bulk-seq Normalized Data
    - a `.csv` file with genes as rows and sample as column
     
    
    |         | Sample  | 
    | :-----: | :-----: | 
    | Gene1   | 5.22    |
    | Gene2   | 3.67    |
    | ...     | ...     |
    | GeneN   | 15.76   |
    
- Single Cell RNA-seq Normalized Data
    - a `.csv` file with genes as rows and cells as columns
- Single Cell RNA-seq Annotation Data
    - a `.csv` file with cell names and celltype annotation columns. The column containing cell names should be named `Cell` and the column containing the labels should be named `Cell_type`
- Spatial Transcriptomics Normalized Data
    - a `.csv` file with genes as rows and cells/spots as columns
- Spatial Transcriptomics Coordinates Data
    - a `.csv` with cell/spot names and coordinates columns. The column containing cell/spot names should be named `Spot` and the column containing the coordinates should be named `xcoord` and `ycoord`


In [6]:
print("loading data......")

# load sc_meta.csv file, containing two columns of cell name and cell type
input_sc_meta = pd.read_csv(input_sc_meta_path, index_col=0)
# load sc_data.csv file, containing gene expression of each cell
input_sc_data = pd.read_csv(input_sc_data_path, index_col=0)
sc_gene = input_sc_data._stat_axis.values.tolist()
# load bulk.csv file, containing one column of gene expression in bulk
input_bulk = pd.read_csv(input_bulk_path, index_col=0)
bulk_gene = input_bulk._stat_axis.values.tolist()
# filter overlapping genes.
intersect_gene = list(set(sc_gene).intersection(set(bulk_gene)))
input_sc_data = input_sc_data.loc[intersect_gene]
input_bulk = input_bulk.loc[intersect_gene]
# load st_meta.csv and st_data.csv, containing coordinates and gene expression of each spot respectively.
input_st_meta = pd.read_csv(input_st_meta_path, index_col=0)
input_st_data = pd.read_csv(input_st_data_path, index_col=0)
print("load data ok")

loading data......
load data ok


input_sc_meta

In [7]:
input_sc_data

,C_417,C_369,C_265,C_450,C_440,C_347,C_463,C_429,C_442,C_430,...,C_269,C_257,C_50,C_304,C_313,C_439,C_363,C_90,C_54,C_289
CRTC2,0.000000,0.000000,0.000000,0.000000,2.231781,0.000000,0.000000,1.312637,0.0,0.000000,...,0.000000,0.0,0.000000,0.0,0.000000,1.775942,0.000000,0.000000,0.00000,0.000000
COPS5,0.000000,0.000000,0.000000,1.948529,0.000000,0.000000,0.000000,1.312637,0.0,0.000000,...,0.000000,0.0,0.000000,0.0,0.000000,0.000000,1.508931,1.571530,0.00000,0.000000
TMF1,0.000000,0.000000,1.439380,0.000000,2.231781,1.389041,0.000000,1.312637,0.0,0.000000,...,0.000000,0.0,0.000000,0.0,0.000000,2.552189,1.508931,1.571530,0.00000,0.000000
OSBPL3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
PSMG4,0.000000,0.000000,1.439380,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ILF2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.090575,0.000000,0.0,0.000000,...,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,1.571530,0.00000,0.000000
ANKRD46,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
SASH1,0.000000,0.000000,0.000000,0.000000,0.000000,1.389041,0.000000,0.000000,0.0,0.000000,...,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
MTRNR2L4,0.000000,1.697653,0.000000,0.000000,2.231781,0.000000,0.000000,1.312637,0.0,0.000000,...,2.198419,0.0,1.554921,0.0,1.923612,1.775942,1.508931,0.000000,0.00000,2.283902


In [8]:
input_st_meta

,Spot,xcoord,ycoord
1,spot_1,1,1
2,spot_2,2,1
3,spot_3,3,1
4,spot_4,4,1
5,spot_5,5,1
6,spot_6,6,1
7,spot_7,1,2
8,spot_8,2,2
9,spot_9,3,2
10,spot_10,4,2


In [9]:
input_st_data

,spot_1,spot_2,spot_3,spot_4,spot_5,spot_6,spot_7,spot_8,spot_9,spot_10,...,spot_21,spot_22,spot_23,spot_24,spot_25,spot_26,spot_27,spot_28,spot_29,spot_30
AAGAB,0.000000,0.000000,0.000000,2.365428,2.365428,0.793433,0.000000,2.337321,4.702748,0.000000,...,0.000000,0.000000,0.793433,0.000000,0.793433,0.000000,0.000000,2.337321,0.000000,0.793433
AAK1,0.000000,0.000000,0.000000,2.556952,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,2.556952,0.000000,0.000000,0.000000,0.000000,2.556952,0.000000,0.000000
AAMP,1.718439,0.000000,1.718439,1.718439,2.057984,4.690727,0.000000,2.057984,0.000000,0.000000,...,1.084092,3.776423,2.851417,0.000000,0.793433,1.839311,3.142075,1.238926,1.839311,2.511872
AASDHPPT,2.525116,0.000000,0.000000,3.477199,0.000000,0.000000,0.952409,1.883208,2.204595,5.538057,...,0.000000,4.866499,4.087803,2.702439,3.157004,0.952409,5.049906,5.227229,4.626074,0.778696
AATF,1.572888,0.000000,1.572888,0.952409,1.572888,1.630440,3.139488,2.187079,3.741098,2.525298,...,0.000000,1.449957,3.080397,2.291141,5.605695,0.952409,2.187079,1.238926,1.449957,1.630440
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
BUD23,1.572888,0.000000,3.503805,2.177013,3.503805,0.793433,0.952409,0.000000,2.291141,2.525298,...,3.015008,1.163867,2.724349,4.222057,5.249647,2.177013,1.084092,1.930916,1.224603,3.888216
NSD2,1.978389,0.000000,1.978389,0.000000,1.978389,0.000000,0.000000,0.000000,2.604381,0.000000,...,2.604381,2.424774,0.000000,0.000000,2.424774,1.978389,0.000000,0.000000,2.604381,0.000000
NSD3,11.510267,6.187656,1.978389,4.434076,5.874775,9.493980,7.769172,6.230352,3.810758,3.326291,...,3.056448,8.722214,2.851417,4.418391,4.191878,10.662594,11.980052,2.028423,5.933309,3.025372
PYM1,2.874875,0.000000,2.874875,3.459819,1.572888,0.000000,1.301986,0.000000,1.449957,1.572888,...,1.301986,2.751944,1.449957,3.459819,3.022846,0.000000,0.000000,0.000000,1.449957,0.000000


### 5. <a id="Marker-used">Calculate marker genes of each celltype</a>


In [10]:
sc = scanpy.AnnData(input_sc_data.T)
sc.obs = input_sc_meta[['Cell_type']]
scanpy.tl.rank_genes_groups(sc, 'Cell_type', method='wilcoxon')
marker_df = pd.DataFrame(sc.uns['rank_genes_groups']['names']).head(args.top_marker_num)
marker_array = np.array(marker_df)
marker_array = np.ravel(marker_array)
marker_array = np.unique(marker_array)
marker = list(marker_array)
sc_marker = input_sc_data.loc[marker, :]
bulk_marker = input_bulk.loc[marker]

### 6. <a id="Data-processing">Data processing</a>

In [11]:
breed = input_sc_meta['Cell_type']
breed_np = breed.values
breed_set = set(breed_np)
id2label = sorted(list(breed_set))  # List of breed
label2id = {label: idx for idx, label in enumerate(id2label)}  # map breed to breed-id

cell2label = dict()  # map cell-name to breed-id
label2cell = defaultdict(set)  # map breed-id to cell-names
for row in input_sc_meta.itertuples():
    cell_name = getattr(row, 'Cell')
    cell_type = label2id[getattr(row, 'Cell_type')]
    cell2label[cell_name] = cell_type
    label2cell[cell_type].add(cell_name)
label_devide_data = dict()
for label, cells in label2cell.items():
    label_devide_data[label] = sc_marker[list(cells)]

single_cell_splitby_breed_np = {}
for key in label_devide_data.keys():
    single_cell_splitby_breed_np[key] = label_devide_data[key].values  # [gene_num, cell_num]
    single_cell_splitby_breed_np[key] = single_cell_splitby_breed_np[key].mean(axis=1)

max_decade = len(single_cell_splitby_breed_np.keys())
single_cell_matrix = []
#
for i in range(max_decade):
    single_cell_matrix.append(single_cell_splitby_breed_np[i].tolist())


single_cell_matrix = np.array(single_cell_matrix)
single_cell_matrix = np.transpose(single_cell_matrix)  # (gene_num, label_num)

bulk_marker = bulk_marker.values  # (gene_num, 1)
bulk_rep = bulk_marker.reshape(bulk_marker.shape[0],)

### 7.  <a id="Celltype-ratio-calculation">Celltype ratio calculation</a>


In [12]:
# calculate celltype ratio in each spot by NNLS
ratio = nnls(single_cell_matrix, bulk_rep)[0]
ratio = ratio/sum(ratio)

ratio_array = np.round(ratio * input_sc_meta.shape[0] * args.ratio_num)
ratio_list = [r for r in ratio_array]

cell_target_num = dict(zip(id2label, ratio_list))

### 8. <a id="Prepare-the-model-input">Prepare the model input</a>



In [13]:
# *********************************************************************
# input：data， celltype， bulk & output: label, dic, single_cell
single_cell = input_sc_data.values.T  # single cell data (600 * 6588)
index_2_gene = (input_sc_data.index).tolist()
breed = input_sc_meta['Cell_type']
breed_np = breed.values
breed_set = set(breed_np)
breed_2_list = list(breed_set)
dic = {}  # breed_set to index {'B cell': 0, 'Monocyte': 1, 'Dendritic cell': 2, 'T cell': 3}
label = []  # the label of cell (with index correspond)
cfg.nclass = len(breed_set)

cfg.ntrain = single_cell.shape[0]
cfg.FeaSize = single_cell.shape[1]
args.feature_size = single_cell.shape[1]
assert cfg.nclass == len(cell_target_num.keys()), "cell type num no match!!!"

for i in range(len(breed_set)):
    dic[breed_2_list[i]] = i
cell = input_sc_meta["Cell"].values

for i in range(cell.shape[0]):
    label.append(dic[breed_np[i]])

label = np.array(label)

# label index the data size of corresponding target
cell_number_target_num = {}
for k, v in cell_target_num.items():
    cell_number_target_num[dic[k]] = v
# *********************************************************************
# generate data by vae
load_model_1 = args.load_model_1
model_choice_1 = args.model_choice_1

### 9. <a id="Model-training/loading">Model training/loading</a>

In [14]:
ratio = -1
if not load_model_1:  # train
    print("begin vae model training...")
    # ********************* training *********************
    net = train_vae(args, single_cell, cfg, label, used_device)
    # ************** training finished *******************
    print("vae training finished!")
else:  # load model
    print("begin vae model loading...")
    net = load_vae(args, cfg, used_device)
    print("vae load finished!")


begin vae model training...


Train Epoch: 9: 100%|██████████| 10/10 [00:22<00:00,  2.23s/it, loss=0.7235, min_loss=0.7235]


min loss = 0.7234522104263306
vae training finished!


### 10. <a id="Data-generation">Data generation</a>


In [15]:
# generate and out put
generate_sc_meta, generate_sc_data = generate_vae(net, args, ratio, single_cell, cfg, label, breed_2_list, index_2_gene, cell_number_target_num, used_device)


Generate Epoch: 3: 100%|██████████| 249/249.0 [00:02<00:00, 90.21it/s] 

generated done!
begin data to spatial mapping...
Data have been prepared...


In [16]:
generate_sc_meta

,Cell,Cell_type
0,C_1,B cell
1,C_2,Monocyte
2,C_3,Monocyte
3,C_4,Monocyte
4,C_5,B cell
...,...,...
244,C_245,Dendritic cell
245,C_246,Dendritic cell
246,C_247,Dendritic cell
247,C_248,Dendritic cell


In [17]:
generate_sc_data

,C_1,C_2,C_3,C_4,C_5,C_6,C_7,C_8,C_9,C_10,...,C_240,C_241,C_242,C_243,C_244,C_245,C_246,C_247,C_248,C_249
CRTC2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
COPS5,0.022782,0.102354,0.109190,0.073801,0.094017,0.089868,0.084254,0.127202,0.023221,0.033330,...,0.037846,0.070905,0.095362,0.128493,0.090498,0.095281,0.042966,0.019896,0.098108,0.050369
TMF1,0.004465,0.000000,0.022806,0.000316,0.000000,0.043806,0.021129,0.000000,0.045134,0.000000,...,0.048472,0.058024,0.004972,0.019848,0.007251,0.015543,0.009396,0.030084,0.000000,0.055206
OSBPL3,0.056206,0.100684,0.150286,0.099528,0.077691,0.033452,0.012955,0.063713,0.074645,0.087051,...,0.102636,0.110074,0.063700,0.114224,0.055141,0.062708,0.071523,0.047878,0.087292,0.050877
PSMG4,0.119247,0.074134,0.115161,0.128651,0.106633,0.127472,0.091330,0.135085,0.093122,0.109497,...,0.101811,0.137316,0.114172,0.140956,0.103543,0.151975,0.149363,0.054837,0.152369,0.092128
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ILF2,0.005733,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.023104,0.000000,0.000000,0.000393,0.000000,0.000000
ANKRD46,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.017445,0.047947,0.033999,0.016198,...,0.046636,0.041075,0.008237,0.008015,0.000000,0.008214,0.000268,0.000000,0.000000,0.000000
SASH1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.020043,0.000000,0.000000,0.000000
MTRNR2L4,0.076325,0.124150,0.090544,0.064659,0.036852,0.110480,0.057916,0.106308,0.087644,0.096577,...,0.119860,0.045829,0.096805,0.067109,0.130635,0.103011,0.107448,0.081901,0.064214,0.066657


### 11. <a id="Data-saving">Data saving</a>

In [18]:
# saving.....
path = osp.join(args.output_path, args.project_name, 'predata')
if not osp.exists(path):
    os.makedirs(path)
name = "vae"
# kl_loss BetaVAE_H
if args.BetaVAE_H:
    name = "BetaVAE"
path_label_generate_csv = os.path.join(path, args.project_name + "_celltype_pred_" + name + "_epoch" + str(args.epoch_num) + '_lr' + str(args.learning_rate) + ".csv")
path_cell_generate_csv = os.path.join(path, args.project_name + "_data_pred_" + name + "_epoch" + str(args.epoch_num) + '_lr' + str(args.learning_rate) + ".csv")

generate_sc_meta.to_csv(path_label_generate_csv)
generate_sc_data.to_csv(path_cell_generate_csv)

print("bulk deconvolution finish!")

print('start to map data to space...')

bulk deconvolution finish!
start to map data to space...


### 12. <a id="Data-Mapping">Mapping generated single cells to spatial locations</a>

After obtaining the generated data, we needed to map each cell and the spot it belongs to.

In [19]:
processer = CreatData(generate_sc_data, generate_sc_meta, input_st_data, args)
processer.cre_data()
runner = Runner(generate_sc_data, generate_sc_meta, input_st_data, input_st_meta, args)
print('start to train the model...')
pred_meta, pred_spot = runner.run()
print("spatial mapping done!")

preparing train data...
sucessfully create positive data
sucessfully create negative data
save xtrain ok！
save ytrain ok！
train data already prepared.
load xtrain ok!
load ytrain ok!
select top 500 marker genes of each cell type...
start to train the model...
saving model done!
model trained sucessfully, start saving output ...
Calculating scores...
Calculating scores done.
save csv ok
spatial mapping done!


Predicted results, including cell, cell type, spot to which cell belongs, spot coordinates, cell coordinates.

In [20]:
pred_meta

,Cell,Cell_type,Spot,Spot_xcoord,Spot_ycoord,Cell_xcoord,Cell_ycoord
0,C_1,Monocyte,spot_1,1,1,1.09,0.93
1,C_2,B cell,spot_1,1,1,0.91,0.76
2,C_3,Monocyte,spot_1,1,1,0.86,0.76
3,C_4,B cell,spot_1,1,1,1.02,0.99
4,C_5,Dendritic cell,spot_1,1,1,1.11,0.94
...,...,...,...,...,...,...,...
292,C_293,Dendritic cell,spot_30,6,5,5.95,4.87
293,C_294,Dendritic cell,spot_30,6,5,6.10,5.02
294,C_295,Dendritic cell,spot_30,6,5,6.14,5.36
295,C_296,B cell,spot_30,6,5,5.64,5.23


Predicted results, cell-gene expression matrix.

In [21]:
pred_spot

,C_1,C_2,C_3,C_4,C_5,C_6,C_7,C_8,C_9,C_10,...,C_288,C_289,C_290,C_291,C_292,C_293,C_294,C_295,C_296,C_297
CRTC2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
COPS5,0.124117,0.000000,0.040262,0.093784,0.108920,0.042966,0.054190,0.071414,0.078336,0.000000,...,0.073140,0.007336,0.082905,0.093784,0.064080,0.108920,0.089044,0.090498,0.054190,0.024777
TMF1,0.053531,0.009156,0.000000,0.000000,0.000000,0.009396,0.001663,0.000000,0.000000,0.046892,...,0.003410,0.054403,0.053841,0.000000,0.000000,0.000000,0.011779,0.007251,0.001663,0.005201
OSBPL3,0.100059,0.056629,0.120081,0.092655,0.054792,0.071523,0.038445,0.073040,0.058551,0.050712,...,0.041425,0.063368,0.094850,0.092655,0.057429,0.054792,0.111327,0.055141,0.038445,0.099124
PSMG4,0.108896,0.142119,0.116834,0.103961,0.184577,0.149363,0.136070,0.084750,0.105285,0.152003,...,0.146239,0.136534,0.102052,0.103961,0.083463,0.184577,0.146020,0.103543,0.136070,0.152656
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ILF2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.022834,0.000000,0.000000,0.000000,...,0.000070,0.068004,0.032051,0.000000,0.000000,0.000000,0.000000,0.023104,0.022834,0.020390
ANKRD46,0.039625,0.000000,0.000000,0.014866,0.000000,0.000268,0.004176,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000796,0.014866,0.000000,0.000000,0.051618,0.000000,0.004176,0.002657
SASH1,0.000000,0.000000,0.000000,0.000000,0.000000,0.020043,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
MTRNR2L4,0.127593,0.094790,0.102483,0.123930,0.146508,0.107448,0.102378,0.041533,0.141296,0.094657,...,0.078486,0.159252,0.106236,0.123930,0.110742,0.146508,0.061946,0.130635,0.102378,0.048887
